In [2]:
import numpy as np
import sys,os,h5py
import random
from random import randint
from capstone_utils.Sampling import get_samples
from capstone_utils.Sampling import check_coords
from capstone_utils.Sampling import define_test
%matplotlib inline
random.seed(a=0)

In [3]:
def define_test(s_test, s_train):
    #2048/16=128
    m=8
    x=random.randint(0,m)*s_train
    y=random.randint(0,m)*s_train
    z=random.randint(0,m)*s_train
    #print(x,y,z)
    return {'x':[x,x+s_test], 'y':[y,y+s_test], 'z':[z,z+s_test]}

def check_coords(test_coords, train_coords):
    valid=True
    for i in ['x','y','z']:
        r=(max(test_coords[i][0], 
               train_coords[i][0]), 
           min(test_coords[i][1],
               train_coords[i][1]))
        if r[0]<=r[1]:
            valid=False
    return valid

In [6]:
def get_samples(s_sample, nsamples, redshift, test_coords):
    #n is size of minibatch, get valid samples (not intersecting with test_coords)
    sample_list=[]
    m=2048-128
    for n in range(nsamples):
        sample_valid=False
        while sample_valid==False:
            x = random.randint(0,m)
            y = random.randint(0,m)
            z = random.randint(0,m)
            sample_coords = {'x':[x,x+s_sample], 
                             'y':[y,y+s_sample], 
                             'z':[z,z+s_sample]}
            
            sample_valid = check_coords(test_coords, 
                                        sample_coords)
        
        sample_list.append(sample_coords)
        
    #Load cube and get samples and convert them to np.arrays
    sample_array=[]
    datapath='../../../../../'
    f = h5py.File(datapath+'fields_z='+redshift+'.hdf5', 'r')
    f=f['delta_HI']
    #f.close()
    for c in sample_list:
        a = f[c['x'][0]:c['x'][1],
              c['y'][0]:c['y'][1],
              c['z'][0]:c['z'][1]]
        
        a = np.array(a)
        sample_array.append(a)
    
    f=0
    return sample_array

In [7]:
# used to generate sample cubes from the whole redshift data

t_coords=define_test(1024,128)
a=get_samples(128,2,'0.0',t_coords)
len(a)
a

OSError: Unable to open file (unable to open file: name = '../../../../../fields_z=0.0.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)